In [1]:
import sys
print(sys.version)

3.10.6 (main, Nov  2 2022, 18:53:38) [GCC 11.3.0]


In [2]:
import sklearn.ensemble
import shap
import numpy as np
import numba
import time
import xgboost

/home/jarek/projects/shap/venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
X,y = shap.datasets.boston()
X.shape



Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np

        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.e.
    :func:`~sklearn.datasets.fetch_california_housing`) and the Ames housing
    datase

(506, 13)

In [4]:
model = sklearn.ensemble.RandomForestRegressor(n_estimators=100, max_depth=6)
model.fit(X, y)



RandomForestRegressor(max_depth=6)

In [5]:
bst = xgboost.train({"learning_rate": 0.01, "max_depth": 4}, xgboost.DMatrix(X, label=y), 1000)

In [6]:
start = time.time()
shap_values = bst.predict(xgboost.DMatrix(X), pred_contribs=True)
print(time.time() - start)


0.08916878700256348


In [7]:
from shap.explainers.pytree import *
# x = np.ones(X.shape[1])
# TreeExplainer(model).shap_values(x)

In [8]:
from shap import TreeExplainer as TreeCext
start = time.time()
t = TreeCext(model).shap_values(X, banz=False)
print(time.time() - start)
t

ile drzew cext? 100
using treeshap and cext
banz = False
here37
using dense_Tree_pred
0.17458176612854004


array([[ 3.05136597e-01,  1.19659216e-03,  1.71649012e-01, ...,
         2.56519958e-01,  1.76545450e-03,  4.76256950e+00],
       [ 3.26084776e-01, -3.04899432e-03, -2.27033157e-02, ...,
         1.28352564e-01,  2.44988969e-02,  1.76324212e+00],
       [ 2.71513763e-01, -7.55095561e-03, -6.49299742e-02, ...,
         1.12008704e-01,  8.95903725e-02,  5.27345897e+00],
       ...,
       [ 2.41741787e-01, -4.46357150e-02, -2.64351490e-01, ...,
        -1.32024603e+00, -5.46288637e-02,  2.96995528e+00],
       [ 3.36753244e-01, -4.77265065e-03, -1.14487465e-01, ...,
        -8.82085349e-01,  4.61444341e-02,  3.88886741e+00],
       [ 2.71348927e-01, -3.29616090e-03, -4.15832967e-02, ...,
        -6.15106804e-01,  2.45745961e-02,  1.31101723e+00]])

In [9]:
start = time.time()
t = TreeCext(model).shap_values(X, banz=True)
print(time.time() - start)
t # TODO BLAD - dodają się zamiast zaczynac od 0 dla kazdej probki danych

ile drzew cext? 100
using treeshap and cext
banz = True
0.22174668312072754
using cext banz


array([[ 3.14589794e-01,  4.91912861e-03,  1.80228361e-01, ...,
         2.53776041e-01, -1.19684646e-03,  4.80901162e+00],
       [ 6.29179587e-01,  9.83825722e-03,  3.60456722e-01, ...,
         5.07552082e-01, -2.39369291e-03,  9.61802325e+00],
       [ 9.43769381e-01,  1.47573858e-02,  5.40685083e-01, ...,
         7.61328124e-01, -3.59053937e-03,  1.44270349e+01],
       ...,
       [ 1.58553256e+02,  2.47924082e+00,  9.08350940e+01, ...,
         1.27903125e+02, -6.03210614e-01,  2.42374186e+03],
       [ 1.58867846e+02,  2.48415995e+00,  9.10153223e+01, ...,
         1.28156901e+02, -6.04407461e-01,  2.42855087e+03],
       [ 1.59182436e+02,  2.48907908e+00,  9.11955507e+01, ...,
         1.28410677e+02, -6.05604307e-01,  2.43335988e+03]])

In [10]:
start = time.time()
ex = TreeExplainer(model)
print(time.time() - start)
start = time.time()
print(ex.shap_values(X))
print(time.time() - start)

0.03382229804992676
oryginalna implemenacja shap_values
other model type
using slow python treeshap
[[ 3.05136597e-01  1.19659216e-03  1.71649012e-01 ...  1.76545450e-03
   4.76256950e+00  2.25226996e+01]
 [ 3.26084776e-01 -3.04899432e-03 -2.27033157e-02 ...  2.44988969e-02
   1.76591470e+00  2.25226996e+01]
 [ 2.71513763e-01 -7.55095561e-03 -6.49299742e-02 ...  8.95903725e-02
   5.27613155e+00  2.25226996e+01]
 ...
 [ 2.41741787e-01 -4.46357150e-02 -2.64351490e-01 ... -5.46288637e-02
   2.96995528e+00  2.25226996e+01]
 [ 3.36753244e-01 -4.77265065e-03 -1.14487465e-01 ...  4.61444341e-02
   3.88886741e+00  2.25226996e+01]
 [ 2.71348927e-01 -3.29616090e-03 -4.15832967e-02 ...  2.45745961e-02
   1.31101723e+00  2.25226996e+01]]
92.03884315490723


In [11]:
start = time.time()
res = ex.banz_values(X)
print(time.time() - start)
res

nasza implementacja banz values
other model type
dwa wymiary
117.3974928855896


[[0.31458979358386063,
  0.004919128610977025,
  0.180228361064573,
  -0.002049986542675222,
  0.0901028065833501,
  -1.2771259451687098,
  -0.03709342942925736,
  -0.1651318927970993,
  -0.086749709742132,
  -0.14403092545993748,
  0.2537760411795978,
  -0.001196846457105063,
  4.8090116249261134],
 [0.35090612626654283,
  -0.0038189352918372166,
  -0.021252335109213535,
  -0.0022416851761767917,
  0.38204418792375866,
  -2.525102896150322,
  -0.023093945133471187,
  -0.2059136035989919,
  -0.010712281096174298,
  0.016444462412949633,
  0.1412112886059991,
  0.02818966405361785,
  1.8133358300574762],
 [0.24107660352425087,
  -0.010601933255672267,
  -0.0693896856145989,
  -0.002568803534336297,
  0.39801445012397324,
  5.936679063428727,
  0.055324990373131204,
  -0.23766543743924767,
  -0.024473866153040955,
  0.28432365827290834,
  0.09699319987437363,
  0.09271379059172193,
  5.184773004674528],
 [0.24680483865336045,
  -0.0020120673372131747,
  0.11493001723232642,
  -0.01382505

In [ ]:
start = time.time()
res = ex.banz_values(X, jit=True)
print(time.time() - start)
res

nasza implementacja banz values
other model type
using numba py banz
dwa wymiary
113.99943447113037
